In [ ]:
import sys
import os
from pathlib import Path
wd = Path().cwd() # working directory
sys.path.insert(1, wd.__str__())

from main.pre_process import combine_tiff
from main.custom_filters import *
from main.quick_deepinterp import *

from ScanImageTiffReader import ScanImageTiffReader
import numpy as np

In [2]:
fld = 'E:\\Code\\ca_denoise\\demo_dataset'
preflix = 'S1ROI1_10.5x_'

In [5]:
combine_tiff(fld, preflix, if_combine = 1)
# from main.pre_process import caiman_mc(fld)
# caiman_mc(fld)
generator_param = quick_finetuning(fld)

loading files
processing
saving files
starting motion correction
starting the cluster


INFO:root:Changing key fnames in group data from None to E:\Code\ca_denoise\demo_dataset\pre\S1ROI1_10.5x__combined.tif
INFO:root:Changing key decay_time in group data from 0.4 to 0.5
INFO:root:Changing key dxy in group data from (1, 1) to (0.5, 0.5)
INFO:root:Changing key pw_rigid in group motion from False to True
INFO:root:Changing key max_shifts in group motion from (6, 6) to [10, 10]
INFO:root:Changing key strides in group motion from (96, 96) to (40, 40)
INFO:root:Changing key overlaps in group motion from (32, 32) to (5, 5)


motion correction for file 0


INFO:root:Generating template by rigid motion correction
INFO:root:** Starting parallel motion correction **
INFO:root:** Finished parallel motion correction **
INFO:root:Saving file as E:\Code\ca_denoise\demo_dataset\pre\S1ROI1_10.5x__combined_els__d1_512_d2_512_d3_1_order_F_frames_1059_.mmap
INFO:root:** Starting parallel motion correction **
INFO:root:** Finished parallel motion correction **
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.21it/s]


creating motion corrected tiff for file 0


...

Starting fine-tuning


INFO:FineTuning:created objects for training


Epoch 1/2
200/200 [==============================] - 264s 1s/step - loss: 0.4557 - val_loss: 0.4361
Epoch 2/2
200/200 [==============================] - 264s 1s/step - loss: 0.4347 - val_loss: 0.4233


INFO:FineTuning:fine tuning job finished - finalizing output model


Saved model to disk
Fine-tuning finished


In [3]:
model_name = '2021_07_31_09_31_03_528689_unet_1024_search_mean_squared_error_pre_30_post_30_feat_32_power_2_depth_4_unet_True-0100-0.5733.h5'
movie_idx = 0
nb_frame_training = 500
input_model_path = os.path.join(wd, "model_folder", model_name)
input_movies = glob.glob(os.path.join(fld, "mc" , "*.tif")) 
input_movie_path = input_movies[0]
generator_param = {}
generator_param["name"] = "SingleTifGenerator"  # Name of object (use SingleTifGenerator for tiff files)
generator_param["pre_frame"] = 30
generator_param["post_frame"] = 30
generator_param["data_path"] = input_movie_path
generator_param["batch_size"] = 1 # This is small because Colab GPUs do have very smaller memory. Increase on better cards. 
generator_param["start_frame"] = 0
generator_param["end_frame"] = -1
generator_param["total_samples"] = nb_frame_training
generator_param["pre_post_omission"] = 0  # Number of frame omitted before and after the predicted frame
m = ScanImageTiffReader(input_movie_path).data()
n_frame = m.shape[0]
# We are reusing the data generator for training here.
generator_param["start_frame"] = 0
generator_param["end_frame"] = -1
generator_param["total_samples"] = n_frame

In [4]:
quick_inference(fld, generator_param)

Preparing data for inference file 0
Starting inference for file 0


INFO:root:randomize should be set to False for inference.                         Overriding the parameter
INFO:Inference:wrote E:\Code\ca_denoise\demo_dataset\denoise\inference_full_args.json
INFO:Inference:wrote E:\Code\ca_denoise\demo_dataset\denoise\2023_06_25_15_50_inference.json
INFO:Inference:wrote E:\Code\ca_denoise\demo_dataset\denoise\2023_06_25_15_50_generator.json
INFO:Inference:created objects for inference


Inference finished for file 0
generate tiff for file 0


TypeError: stat: path should be string, bytes, os.PathLike or integer, not tuple

In [14]:
def h5_2_tiff(out_file, orig_file, generator_param):
    import numpy as np
    import tifffile
#    import pytiff
    import cv2
    
    data = np.asarray(h5py.File(out_file,'r')['data'])
    data = cv2.normalize(data, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    
    # orig_data = []
    # with pytiff.Tiff(orig_file, 'r') as handle:
    #     for page in handle.pages:
    #         orig_data.append(np.asarray(page[:]))
    # orig_data = np.asarray(orig_data)
    from ScanImageTiffReader import ScanImageTiffReader
    orig_data = ScanImageTiffReader(orig_file).data()
    orig_data = cv2.normalize(orig_data, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    
    data[:generator_param["pre_frame"],:] = orig_data[:generator_param["pre_frame"],:]
    data[-generator_param["post_frame"]:,:] = orig_data[-generator_param["post_frame"]:,:]
    basename = os.path.splitext(os.path.basename(out_file))[0]
    final_dir = os.path.join(os.path.dirname(os.path.dirname(out_file)), "final")
    if not os.path.exists(final_dir):
        os.makedirs(final_dir)
    tifffile.imsave(os.path.join(final_dir, basename) + '.tif',data)

In [15]:
output_dir = os.path.join(fld, "denoise")
file = glob.glob(os.path.join(fld, "mc" , "*.tif"))[0]
base_file = os.path.splitext(os.path.basename(file))[0]
outfile = os.path.join(output_dir, base_file+'_denoise.h5')

h5_2_tiff(outfile, file, generator_param)

In [13]:
final_dir = os.path.join(os.path.dirname(os.path.dirname(outfile)), "final")
basename = os.path.splitext(os.path.basename(outfile))[0]

os.path.join(final_dir, basename) + '.tif'

'E:\\Code\\ca_denoise\\demo_dataset\\final\\S1ROI1_10.5x__combined_corrected_denoise.tif'